# import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.executable

'/users/samova/lcances/.miniconda3/envs/pytorch-dev/bin/python'

In [3]:
import os
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["NUMEXPR_NU M_THREADS"] = "2"
os.environ["OMP_NUM_THREADS"] = "2"
import time
from typing import Union

import numpy
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F
import torchvision.transforms as transforms

from torch.cuda.amp import autocast
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader

# from torchlibrosa.augmentation import SpecAugmentation

In [4]:
from SSL.util.model_loader import load_model
from SSL.util.loaders import load_dataset, load_optimizer, load_callbacks, load_preprocesser
from SSL.util.checkpoint import CheckPoint, mSummaryWriter
from SSL.util.utils import reset_seed, get_datetime, track_maximum, DotDict
from SSL.util.mixup import MixUpBatchShuffle

from metric_utils.metrics import BinaryAccuracy, FScore, ContinueAverage

# Arguments

In [5]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--from_config", default="", type=str)
parser.add_argument("-d", "--dataset_root", default="../datasets", type=str)
parser.add_argument("-D", "--dataset", default="audioset-unbalanced", type=str)

group_t = parser.add_argument_group("Commun parameters")
group_t.add_argument("-m", "--model", default="wideresnet28_2", type=str)
group_t.add_argument("--supervised_ratio", default=1.0, type=float)
group_t.add_argument("--batch_size", default=128, type=int)
group_t.add_argument("--nb_epoch", default=500_000, type=int) # nb iteration
group_t.add_argument("--learning_rate", default=0.003, type=float)
group_t.add_argument("--resume", action="store_true", default=False)
group_t.add_argument("--seed", default=1234, type=int)

group_mixup = parser.add_argument_group("Mixup parameters")
group_mixup.add_argument("--mixup", action="store_true", default=False)
group_mixup.add_argument("--mixup_alpha", type=float, default=0.4)
group_mixup.add_argument("--mixup_max", action="store_true", default=False)
group_mixup.add_argument("--mixup_label", action="store_true", default=False)

group_sa = parser.add_argument_group("Spec augmentation")
group_sa.add_argument("--specAugment", action="store_true", default=False)
group_sa.add_argument("--sa_time_drop_width", type=int, default=32)
group_sa.add_argument('--sa_time_stripes_num', type=int, default=1)
group_sa.add_argument("--sa_freq_drop_width", type=int, default=4)
group_sa.add_argument("--sa_freq_stripes_num", type=int, default=1)

group_u = parser.add_argument_group("Datasets parameters")
group_u.add_argument("-t", "--train_folds", nargs="+", default=[1, 2, 3, 4], type=int)
group_u.add_argument("-v", "--val_folds", nargs="+", default=[5], type=int)

group_l = parser.add_argument_group("Logs")
group_l.add_argument("--checkpoint_root", default="../model_save/", type=str)
group_l.add_argument("--tensorboard_root", default="../tensorboard/", type=str)
group_l.add_argument("--checkpoint_path", default="supervised", type=str)
group_l.add_argument("--tensorboard_path", default="supervised", type=str)
group_l.add_argument("--log_suffix", default="", type=str)

parser.add_argument("-N", "--nb_gpu", default=1, type=int)


args = parser.parse_args(['--nb_gpu', '2'])

args.tensorboard_path = os.path.join(args.tensorboard_root, args.dataset, args.tensorboard_path)
args.checkpoint_path = os.path.join(args.checkpoint_root, args.dataset, args.checkpoint_path)

In [6]:
vars(args)

{'from_config': '',
 'dataset_root': '../datasets',
 'dataset': 'audioset-unbalanced',
 'model': 'wideresnet28_2',
 'supervised_ratio': 1.0,
 'batch_size': 128,
 'nb_epoch': 500000,
 'learning_rate': 0.003,
 'resume': False,
 'seed': 1234,
 'mixup': False,
 'mixup_alpha': 0.4,
 'mixup_max': False,
 'mixup_label': False,
 'specAugment': False,
 'sa_time_drop_width': 32,
 'sa_time_stripes_num': 1,
 'sa_freq_drop_width': 4,
 'sa_freq_stripes_num': 1,
 'train_folds': [1, 2, 3, 4],
 'val_folds': [5],
 'checkpoint_root': '../model_save/',
 'tensorboard_root': '../tensorboard/',
 'checkpoint_path': '../model_save/audioset-unbalanced/supervised',
 'tensorboard_path': '../tensorboard/audioset-unbalanced/supervised',
 'log_suffix': '',
 'nb_gpu': 2}

# initialisation

In [7]:
reset_seed(args.seed)


trainer = SupervisedTrainer("cnn03", "esc10")
trainer.init_trainer(
    parameters=vars(args),
    seed = args.seed,
    num_workers=0,
    pin_memory=True,
    verbose = 2
)

In [8]:
# from SSL.trainers.esc import SupervisedTrainer
from SSL.trainers.trainers import Trainer

class SupervisedTrainer(Trainer):
    
    def __init__(self, model: str, dataset: str):
        super().__init__(model, "supervised", dataset)

trainer = SupervisedTrainer(args.model, args.dataset)

# Prepare the dataset

In [9]:
trainer.load_transforms()

Load the transformation


/usr/local/PyTorch/audio/torchaudio/extension/extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [10]:
parameters = dict(
    dataset=args.dataset,

    dataset_root = args.dataset_root,
    supervised_ratio = args.supervised_ratio,
    batch_size = args.batch_size,
    train_folds = args.train_folds,
    val_folds = args.val_folds,
    
    num_workers=10,
    pin_memory=True,

    verbose = 2,
)

trainer.load_dataset(parameters)

Load the dataset
/usr/local/PyTorch/audio/torchaudio/datasets/speechcommands.py
______
eva
['eval.h5']
______
unb
['unbalanced_train_part08.h5', 'unbalanced_train_part02.h5', 'unbalanced_train_part05.h5', 'unbalanced_train_part10.h5', 'unbalanced_train_part01.h5', 'unbalanced_train_part03.h5', 'unbalanced_train_part07.h5', 'unbalanced_train_part06.h5', 'unbalanced_train_part09.h5', 'unbalanced_train_part04.h5']
Getting all target
Sort the classes


# Prep model

In [11]:
from types import MethodType
from torch.cuda import empty_cache
from torchsummary import summary


def create_model(self, nb_gpu: int = 1):
    print("Create the model")
    empty_cache()

    model_func = load_model(self.dataset, self.model_str)
    self.model = model_func(
        input_shape=self.input_shape,
        num_classes=self.num_classes,
    )
    self.model = self.model.cuda()
    
    s = summary(self.model, self.input_shape)
    
    if nb_gpu > 1:
        self.model = nn.DataParallel(self.model)
    
trainer.create_model = MethodType(create_model, trainer)
trainer.create_model(args.nb_gpu)

Create the model
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 32, 64, 500]             288
       BatchNorm2d-2          [-1, 32, 64, 500]              64
              ReLU-3          [-1, 32, 64, 500]               0
         MaxPool2d-4          [-1, 32, 32, 250]               0
            Conv2d-5          [-1, 32, 32, 250]           9,216
       BatchNorm2d-6          [-1, 32, 32, 250]              64
              ReLU-7          [-1, 32, 32, 250]               0
            Conv2d-8          [-1, 32, 32, 250]           9,216
       BatchNorm2d-9          [-1, 32, 32, 250]              64
             ReLU-10          [-1, 32, 32, 250]               0
       BasicBlock-11          [-1, 32, 32, 250]               0
           Conv2d-12          [-1, 32, 32, 250]           9,216
      BatchNorm2d-13          [-1, 32, 32, 250]              64
             ReLU-14  

# Training initialization

## Losses

In [12]:
def init_loss(self):
    self.loss_ce = nn.BCEWithLogitsLoss(reduction="mean")

trainer.init_loss = MethodType(init_loss, trainer)

In [13]:
trainer.init_loss()

## optimizer & callbacks

In [14]:
parameters=DotDict(
    learning_rate=args.learning_rate,
)
trainer.init_optimizer(parameters)


Initialize optimizer


In [15]:
parameters=DotDict(
    nb_epoch=args.nb_epoch,
    optimizer=trainer.optimizer,
)
trainer.init_callbacks(parameters)

Initialize callbacks


# Logs and checkpoint

In [16]:
# Prepare suffix
# normale training parameters
sufix_title = ''
sufix_title += f'_{args.learning_rate}-lr'
sufix_title += f'_{args.supervised_ratio}-sr'
sufix_title += f'_{args.nb_epoch}-e'
sufix_title += f'_{args.batch_size}-bs'
sufix_title += f'_{args.seed}-seed'

# mixup parameters
if args.mixup:
    sufix_title += '_mixup'
    if args.mixup_max: sufix_title += "-max"
    if args.mixup_label: sufix_title += "-label"
    sufix_title += f"-{args.mixup_alpha}-a"
    
# SpecAugment parameters
if args.specAugment:
    sufix_title += '_specAugment'
    sufix_title += f'-{args.sa_time_drop_width}tdw'
    sufix_title += f'-{args.sa_time_stripes_num}tsn'
    sufix_title += f'-{args.sa_freq_drop_width}fdw'
    sufix_title += f'-{args.sa_freq_stripes_num}fsn'

In [17]:
sufix_title

'_0.003-lr_1.0-sr_500000-e_128-bs_1234-seed'

In [18]:
# Logs
parameters=DotDict(
    supervised_ratio=args.supervised_ratio
)

trainer.init_logs(parameters, suffix=sufix_title)

Prepare the log system


In [19]:
# Checkpoint
parameters=DotDict(
    supervised_ratio=args.supervised_ratio
)
trainer.init_checkpoint(parameters, suffix=sufix_title)

Prepare the checkpoint system
checkpoint initialise at:  /users/samova/lcances/semi-supervised/model_save/audioset-unbalanced/supervised/wideresnet28_2/1.0S/wideresnet28_2_1.0S_0.003-lr_1.0-sr_500000-e_128-bs_1234-seed
name:  wideresnet28_2_1.0S_0.003-lr_1.0-sr_500000-e_128-bs_1234-seed
mode:  max


## Metrics

In [20]:
# Metrics
from metric_utils.metrics import Metrics
from sklearn import metrics


class MAP(Metrics):
    def __init__(self, epsilon=1e-10):
        super().__init__(epsilon)

    def __call__(self, y_pred, y_true):
        super().__call__(y_pred, y_true)
        aps = metrics.average_precision_score(y_true, y_pred, average=None)
        aps = numpy.nan_to_num(aps)
        
        self.values.append(aps.mean())
        return self

def init_metrics(self):
    self.metrics = DotDict(
        fscore_fn=FScore(),
        acc_fn=BinaryAccuracy(),
        avg_fn=ContinueAverage(),
        mAP_fn=MAP()
    )
    self.time_average_fn = ContinueAverage()
    
    self.maximum_tracker = track_maximum()

trainer.init_metrics = MethodType(init_metrics, trainer)
trainer.init_metrics()

## training function

In [21]:
def set_printing_form(self):
    UNDERLINE_SEQ = "\033[1;4m"
    RESET_SEQ = "\033[0m"

    header_form = "Type            Epoch -       /       - Losses: bce       - Metrics: acc         F1           mAP           - Remaining time "
    header_form = "{:<16.16} {:<5.5} - {:<5.5} / {:<5.5} - {:<7.7} {:<9.9} - {:<8.8} {:<12.12} {:<12.12} {:<12.12} - {:<6.6}"
    value_form  = "{:<16.16} {:<5} - {:>5} / {:<5} - {:7.7} {:<9.4f} - {:<8.8} {:<12.3e} {:<12.3e} {:<12.3e} - {:<6.4f}"

    self.header = header_form.format(
        ".               ", "Epoch", "", "", "Losses:", "ce", "metrics: ", "acc", "F1", "mAP", "Time"
    )

    self.train_form = value_form
    self.val_form = UNDERLINE_SEQ + value_form + RESET_SEQ
    
trainer.set_printing_form = MethodType(set_printing_form, trainer)

# init mixup and SpecAugment

In [22]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F


class DropStripes(nn.Module):
    def __init__(self, dim, drop_width, stripes_num):
        """Drop stripes. 
        Args:
          dim: int, dimension along which to drop
          drop_width: int, maximum width of stripes to drop
          stripes_num: int, how many stripes to drop
        """
        super(DropStripes, self).__init__()

        assert dim in [1, 2]    # dim 2: frequency; dim 3: time

        self.dim = dim
        self.drop_width = drop_width
        self.stripes_num = stripes_num

    def forward(self, input):
        """input: (batch_size, channels, time_steps, freq_bins)"""

        assert input.ndimension() == 3

        if self.training is False:
            return input

        else:
            batch_size = input.shape[0]
            total_width = input.shape[self.dim]

            for n in range(batch_size):
                self.transform_slice(input[n], total_width)

            return input


    def transform_slice(self, e, total_width):
        """e: (channels, time_steps, freq_bins)"""

        for _ in range(self.stripes_num):
            distance = torch.randint(low=0, high=self.drop_width, size=(1,))[0]
            bgn = torch.randint(low=0, high=total_width - distance, size=(1,))[0]

            if self.dim == 1:
                e[bgn : bgn + distance, :] = 0
            elif self.dim == 2:
                e[:, bgn : bgn + distance] = 0


class SpecAugmentation(nn.Module):
    def __init__(self, time_drop_width, time_stripes_num, freq_drop_width, 
        freq_stripes_num):
        """Spec augmetation. 
        [ref] Park, D.S., Chan, W., Zhang, Y., Chiu, C.C., Zoph, B., Cubuk, E.D. 
        and Le, Q.V., 2019. Specaugment: A simple data augmentation method 
        for automatic speech recognition. arXiv preprint arXiv:1904.08779.
        Args:
          time_drop_width: int
          time_stripes_num: int
          freq_drop_width: int
          freq_stripes_num: int
        """

        super(SpecAugmentation, self).__init__()

        self.time_dropper = DropStripes(dim=2, drop_width=time_drop_width, 
            stripes_num=time_stripes_num)

        self.freq_dropper = DropStripes(dim=1, drop_width=freq_drop_width, 
            stripes_num=freq_stripes_num)

    def forward(self, input):
        x = self.time_dropper(input)
        x = self.freq_dropper(x)
        return x

In [23]:
# # Spec augmenter
spec_augmenter = SpecAugmentation(time_drop_width=args.sa_time_drop_width,
                                  time_stripes_num=args.sa_time_stripes_num,
                                  freq_drop_width=args.sa_freq_drop_width,
                                  freq_stripes_num=args.sa_freq_stripes_num)

# Mixup
mixup_fn = MixUpBatchShuffle(alpha=args.mixup_alpha, apply_max=args.mixup_max, mix_labels=args.mixup_label)

In [24]:
batch_summed = []


# def calc_class_dist(y):
#     with torch.set_grad_enabled(False):
#         summed = torch.sum(y, axis=0)
#         summed = summed[summed > 0]
#         if len(summed) <= 0:
#             return False

#         ratio = min(summed) / max(summed)
#         if ratio < 0.16:
#             return False
        
#         return True

def train_fn(self, epoch, X, y, start_time) -> Union[float, float]:
    # aliases
    M = self.metrics
    T = self.tensorboard.add_scalar

    self.model.train()

    y_ = y.detach().clone() # keep a copy outside the graph and in cpu to compute the mAP

    X = X.cuda().float()
    y = y.cuda().float()

    if args.mixup:
        X, y = mixup_fn(X, y)

    if args.specAugment:
        X = spec_augmenter(X)

    logits = self.model(X)
    loss = self.loss_ce(logits, y)

    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

    with torch.set_grad_enabled(False):

        pred = torch.sigmoid(logits)

        fscore = M.fscore_fn(pred, y)
        acc = M.acc_fn(pred, y)
        avg_ce = M.avg_fn(loss.item())

        end_time = time.time()
        running_mean_time = self.time_average_fn(end_time - start_time)

        # logs
        print(self.train_form.format(
            "Training: ",
            epoch + 1,
            e, args.nb_epoch,
            "", avg_ce.mean(size=1000),
            "", acc.mean(size=1000), fscore.mean(size=1000), 0.0,
            time.time() - start_time,
        ), end="\r")

    T("train/Lce", avg_ce.mean(size=1000), epoch)
    T("train/f1", fscore.mean(size=1000), epoch)
    T("train/acc", acc.mean(size=1000), epoch)
    
    return avg_ce.value, fscore.value

trainer.train_fn = MethodType(train_fn, trainer)

In [25]:
def val_fn(self, epoch: int)  -> Union[float, float]:
    # aliases
    M = self.metrics
    T = self.tensorboard.add_scalar
    nb_batch = len(self.val_loader)

    start_time = time.time()
    print("")

    self.reset_metrics()
    self.model.eval()

    with torch.set_grad_enabled(False):
        for i, (X, y) in enumerate(self.val_loader):
            X = X.cuda().float()
            y = y.cuda().float()

            logits = self.model(X)
            loss = self.loss_ce(logits, y)

            pred = torch.sigmoid(logits)
            fscore = M.fscore_fn(pred, y)            
            acc = M.acc_fn(pred, y)
            mAP = M.mAP_fn(pred.cpu().reshape(-1), y.cpu().reshape(-1))
            avg_ce = M.avg_fn(loss.item())

            # logs
            print(self.val_form.format(
                "Validation: ",
                epoch + 1,
                i, nb_batch,
                "", avg_ce.mean(size=1000),
                "", acc.mean(size=1000), fscore.mean(size=1000), 0.0,
                time.time() - start_time
            ), end="\r")

    T("val/Lce", avg_ce.mean(size=1000), epoch)
    T("val/f1", fscore.mean(size=1000), epoch)
    T("val/acc", acc.mean(size=1000), epoch)

    T("hyperparameters/learning_rate", self._get_lr(), epoch)

    T("max/acc", self.maximum_tracker("acc", acc.mean(size=1000)), epoch)
    T("max/f1", self.maximum_tracker("f1", fscore.mean(size=1000)), epoch)
    
    return avg_ce, fscore
    
trainer.val_fn = MethodType(val_fn, trainer)

In [26]:
def test_fn(self):
    # aliases
    M = self.metrics
    T = self.tensorboard.add_scalar
    nb_batch = len(self.val_loader)

    # Load best epoch
    self.checkpoint.load_best()

    start_time = time.time()
    print("")

    self.reset_metrics()
    self.model.eval()

    with torch.set_grad_enabled(False):
        for i, (X, y) in enumerate(self.test_loader):
            X = X.cuda()
            y = y.cuda()

            logits = self.model(X)
            loss = self.loss_ce(logits, y)

            pred = torch.sigmoid(logits)
            y_one_hot = y # F.one_hot(y, num_classes=self.num_classes)
            fscore = M.fscore_fn(pred, y_one_hot).mean
            
            acc = M.acc_fn(logits, y).mean
            
            avg_ce = M.avg_fn(loss.item()).mean

            # logs
            print(self.val_form.format(
                "Testing: ",
                1,
                i, nb_batch,
                "", avg_ce,
                "", acc, fscore,
                time.time() - start_time
            ), end="\r")
            
    return avg_ce, fscore
            
trainer.test_fn = MethodType(test_fn, trainer)

# Training

In [27]:
# Resume if wish
if args.resume:
    trainer.checkpoint.load_last()

In [ ]:
# Fit function
trainer.set_printing_form()
print(trainer.header)

start_epoch = trainer.checkpoint.epoch_counter
end_epoch = args.nb_epoch

train_iterator = iter(trainer.train_loader)
start_time = time.time()

for e in range(start_epoch, args.nb_epoch):
    # Perform train 
    train_avg_ce, train_fscore = trainer.train_fn(e, *train_iterator.next(), start_time)
    
    # Validation every 10 000 iteration
    if e % 10_000 == 0 and e != 0:
        val_avg_ce, val_fscore = trainer.val_fn(e)
        trainer.checkpoint.step(val_fscore.value)
    
    # Apply the different callbacks
#     for c in trainer.callbacks:
#         c.step()
    
    if e % 1000 == 0:
        trainer.tensorboard.flush()
    
trainer.save_hparams(vars(args))

.                Epoch -       /       - Losses: ce        - metrics: acc          F1           mAP          - Time  
Training:        10001 - 10000 / 500000 -         0.0101    -          9.977e-01    5.051e-01    0.000e+00    - 541.5352


/users/samova/lcances/.miniconda3/envs/pytorch-dev/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


Training:        20001 - 20000 / 500000 -         0.0048    -          9.986e-01    6.425e-01    0.000e+00    - 1586.63404227
Training:        30001 - 30000 / 500000 -         0.0010    -          9.998e-01    8.386e-01    0.000e+00    - 2637.79088095
Training:        40001 - 40000 / 500000 -         0.0002    -          1.000e+00    8.790e-01    0.000e+00    - 3699.55829828
Training:        50001 - 50000 / 500000 -         0.0001    -          1.000e+00    8.820e-01    0.000e+00    - 4759.18136560
Training:        60001 - 60000 / 500000 -         0.0001    -          1.000e+00    8.830e-01    0.000e+00    - 5817.22918772
Training:        70001 - 70000 / 500000 -         0.0000    -          1.000e+00    8.840e-01    0.000e+00    - 6886.31809629
Training:        80001 - 80000 / 500000 -         0.0000    -          1.000e+00    8.840e-01    0.000e+00    - 7954.36931027
Training:        90001 - 90000 / 500000 -         0.0000    -          1.000e+00    8.810e-01    0.000e+00    - 9027.5

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/users/samova/lcances/.miniconda3/envs/pytorch-dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-579e21b497a8>", line 17, in <module>
    val_avg_ce, val_fscore = trainer.val_fn(e)
  File "<ipython-input-25-5e618d7d88eb>", line 33, in val_fn
    "", acc.mean(size=1000), fscore.mean(size=1000), 0.0,
  File "/users/samova/lcances/pytorch_metrics/metric_utils/metrics.py", line 28, in mean
    accumulate = sum(self.values[-size:])
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/users/samova/lcances/.miniconda3/envs/pytorch-dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handlin

In [ ]:
batch_summed_ = batch_summed.copy()
ratio = []

for i in range(len(batch_summed)):
    batch = batch_summed_[i].cpu()
    batch = batch[batch > 0]
    
    ratio.append(min(batch) / max(batch))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(0, figsize=(20,5))
plt.plot(ratio)

plt.show()

In [ ]:
if args.dataset.lower() == "speechcommand":
    trainer.test_fn()

# ♫♪.ılılıll|̲̅̅●̲̅̅|̲̅̅=̲̅̅|̲̅̅●̲̅̅|llılılı.♫♪

# Compute mAP

# Available models

In [67]:
path_root = '/users/samova/lcances/semi-supervised/model_save/audioset-unbalanced/supervised'
model_dir = os.listdir(path_root)

for model in model_dir:
    print('')
    print('='*8, model, "="*8)
    model_path = os.path.join(path_root, model)
    
    ratio_dir = os.listdir(model_path)
    for ratio in ratio_dir:
        ratio_path = os.path.join(model_path, ratio)
        model_list = os.listdir(ratio_path)
        
        for model_save in model_list:
            final_path = os.path.join(ratio_path, model_save)
            print('\t', final_path)
            


======== wideresnet28_2 ========
	 /users/samova/lcances/semi-supervised/model_save/audioset-unbalanced/supervised/wideresnet28_2/0.5S/wideresnet28_2_0.5S.last
	 /users/samova/lcances/semi-supervised/model_save/audioset-unbalanced/supervised/wideresnet28_2/1.0S/wideresnet28_2_1.0S_0.003-lr_1.0-sr_15-e_128-bs_1234-seed_mixup-label-0.4-a_specAugment-32tdw-1tsn-4fdw-1fsn.last
	 /users/samova/lcances/semi-supervised/model_save/audioset-unbalanced/supervised/wideresnet28_2/1.0S/wideresnet28_2_1.0S_0.003-lr_1.0-sr_15-e_128-bs_1234-seed.last
	 /users/samova/lcances/semi-supervised/model_save/audioset-unbalanced/supervised/wideresnet28_2/1.0S/wideresnet28_2_1.0S_0.003-lr_1.0-sr_15-e_128-bs_1234-seed_specAugment-32tdw-1tsn-4fdw-1fsn.last

======== wideresnet28_8 ========
	 /users/samova/lcances/semi-supervised/model_save/audioset-unbalanced/supervised/wideresnet28_8/1.0S/wideresnet28_8_1.0S.last

======== MobileNetV1 ========
	 /users/samova/lcances/semi-supervised/model_save/audioset-unbalanc

In [68]:
trainer.checkpoint.load('/users/samova/lcances/semi-supervised/model_save/audioset-unbalanced/supervised/cnn14/1.0S/cnn14_1.0S_0.003-lr_1.0-sr_15-e_128-bs_1234-seed_mixup-max-label-1.0-a.last')

KeyboardInterrupt: 

In [61]:
from metric_utils.metrics import Precision, Recall

total_pred = []
total_targets = []

trainer.model.eval()

nb_batch = len(trainer.val_loader)

S = nn.Sigmoid()

with torch.set_grad_enabled(False):
    for i, (X, y) in enumerate(trainer.val_loader):
        X = X.cuda().float()
        y = y.cuda().float()

        logits = trainer.model(X)
        
        total_pred.append(S(logits).cpu())
        total_targets.append(y.cpu())
        
        print("%d / %d" % (i, nb_batch), end="\r")

/users/samova/lcances/.miniconda3/envs/pytorch-dev/lib/python3.8/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 5, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [62]:
total_pred_ = numpy.vstack(total_pred)
total_targets_ = numpy.vstack(total_targets)

In [63]:
len(total_targets_), len(total_pred)

(20352, 159)

In [64]:
from sklearn import metrics
import numpy as np
import tqdm

In [65]:
mAP = metrics.average_precision_score(total_targets_, total_pred_, average=None)

In [66]:
mAP.mean()

0.005341823262602574

# Computing mAUC

In [50]:
total_targets = numpy.vstack(total_targets)
total_pred = numpy.vstack(total_pred)

In [51]:
from sklearn import metrics

metrics_auc = []
for i in tqdm.tqdm(range(527)):
    y = total_targets[:,i]
    pred = total_pred[:,i]

    fpr, tpr, thresholds = metrics.roc_curve(y, pred)
    metrics_auc.append(metrics.auc(fpr, tpr))

100%|██████████| 527/527 [00:01<00:00, 281.57it/s]


In [52]:
numpy.mean(metrics_auc)


0.51428088034538

# Computing d-prime
https://stats.stackexchange.com/questions/492673/understanding-and-implementing-the-dprime-measure-in-python

In [ ]:
from scipy.stats import norm
Z = norm.ppf

def calc_dprime(y_true, y_pred):
    return numpy.sqrt(2) * Z(metrics.roc_auc_score(y_true,y_pred))

dprimes = []
for i in tqdm.tqdm(range(527)):
    y = total_targets[:,i]
    pred = total_pred[:,i]

    dprimes.append(calc_dprime(y, pred))

In [ ]:
numpy.mean(dprimes)

# Testing `fit` function from trainer.

In [ ]:
from SSL.trainers import SupervisedTrainer

training_params=dict(
    dataset=args.dataset,

    dataset_root = args.dataset_root,
    supervised_ratio = args.supervised_ratio,
    batch_size = args.batch_size,
    train_folds = args.train_folds,
    val_folds = args.val_folds,
    learning_rate=args.learning_rate,
    nb_epoch=args.nb_epoch,
    seed=args.seed,

)
other_params = dict(
    num_workers=0,
    pin_memory=True,
    verbose = 2,
)

trainer = SupervisedTrainer("cnn03", "esc10")
trainer.init_trainer(
    training_params,
    **other_params
    
)

In [ ]:
trainer.fit()

# Cross validation

In [ ]:
trainer = SupervisedTrainer("wideresnet28_2", "esc10")

train_folds = [[1, 2, 3, 4],
               [2, 3, 4, 5],
               [3, 4, 5, 1],
               [4, 5, 1, 2],
               [5, 1, 2, 3]]
val_folds = [[5], [1], [2], [3], [4]]

for tf, vf in zip(train_folds, val_folds):
    training_params["train_folds"] = tf
    training_params["val_folds"] = vf

    trainer.init_trainer(
        training_params,
        **other_params
    )

    trainer.fit()
    trainer.close()

# Grid search seed

In [ ]:
import numpy as np
seeds = np.random.randint(10000, size=(1))
history = {}

trainer = SupervisedTrainer("wideresnet28_2", "esc10")

train_folds = [1, 2, 3, 4]
val_folds = [5]
seeds = np.random

for seed in seeds:
    training_params["train_folds"] = [1, 2, 3, 4]
    training_params["val_folds"] = [5]
    training_params["seed"] = seed

    trainer.init_trainer(
        training_params,
        **other_params
    )

    trainer.fit()

    history[seed] = trainer.maximum_tracker
    trainer.close()